In [1]:
%%writefile stripped_detector_utility.py
#%%writefile MUST be the first line of the notebook to write it to a .py file
# This will write a python file to be used in other files (e.g. from spont_syn_detector_utility import * == aka from this file

"""This utility function houses all the functions that we will use for correcting baseline fluorescence,
calculating peak times, and calculating amplitudes. This file is the basis of all further calculations that are done
Therefore, if something is wrong here, everything downstream will fail"""


import pandas as pd
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

from scipy.signal import find_peaks, peak_prominences #peak prominences was used in trouble shooting, will not remove for now
from BaselineRemoval import BaselineRemoval

rms_noise = 8.916729622552639
"""#rms_noise was impirically established for this experiment, it might be variable and a function to automate this might 
be needed
 FOR CAMIN GCAMP6 SYNAPSES """

def single_synapse_peak_plotting(input_f, input_fneu, rms_noise = rms_noise):
    """This function is simply used for troubleshooting, this will quickly give you a readout
    of the fluorescence output of each trace from a run of suite2p"""
    corrected = input_f - (0.7*input_fneu)
        # a standard subtraction of fluorescence background immediately surrounding each ROI
    corrected = BaselineRemoval(corrected)
    corrected = corrected.ZhangFit(repitition = 100)
        # an adaptively weighted iterated modified polynomial fit that ignores peaks and corrects baseline to 0
    threshold = np.median(corrected) + rms_noise
    mini_peaks, _ = find_peaks(corrected, height = threshold)
        # scipy find_peaks function
        #then plot the traces you generate
    plt.plot(corrected)
    plt.plot(mini_peaks, corrected[mini_peaks], "x")
    plt.plot(np.full_like(corrected, threshold), "--",color = "grey")
    plt.plot(np.full_like(corrected, np.median(corrected)), "--", color = 'r')
    plt.show()



def single_synapse_baseline_correction_and_peak_return(input_f, input_fneu, rms_noise = rms_noise, return_peaks = True):
    """This function is the one that we will utilize in the pipeline to create a list of peak times
    and their corresponding amplitudes. To return peak times, simply input a single f / fneu file and define rms_noise
    before you enter into the pipeline
    To make this function iterate over a F.npy and Fneu.npy file call the following:
    
        for x, y in zip(F, Fneu):
            single_synapse_baseline_correction_and_peak_return(x,y,rms_noise) 
            
        To return amplitudes, set 'return_peak' = False"""
    
    corrected_trace = input_f - (0.7*input_fneu)
    corrected_trace = BaselineRemoval(corrected_trace)
    corrected_trace = corrected_trace.ZhangFit(repitition = 100)
    threshold = np.median(corrected_trace) + rms_noise
    cell_peaks, _ = find_peaks(corrected_trace, height = threshold)
    amplitudes = corrected_trace[cell_peaks] - np.median(corrected_trace)
    
    if return_peaks == True:
        return cell_peaks
    else:
        return amplitudes

# """ please go from this file to the suite2p_utility notebook to see how this will be used within Suite2p's output """ 
    


Overwriting stripped_detector_utility.py
